In [1]:
import sys

from os import getcwd
from os.path import dirname, abspath


sys.path.append(dirname((abspath(getcwd()))))

In [2]:
import numpy as np
import pandas as pd

from joblib import load

from school_budget.config import LABELS
from school_budget.data import multilabel_sample_dataframe, multilabel_train_test_split
from school_budget.models import predict, top_n_predictions_ids, format_predictions

In [3]:
df = pd.read_csv("../datasets/TrainingData.csv", index_col=0)
NON_LABELS = [c for c in df.columns if c not in LABELS]
SAMPLE_SIZE = 40000
sampling = multilabel_sample_dataframe(
    df,
    pd.get_dummies(df[LABELS]),
    size=SAMPLE_SIZE,
    min_count=25,
    seed=43
)
dummy_labels = pd.get_dummies(sampling[LABELS])
X_train, X_test, y_train, y_test = multilabel_train_test_split(
    sampling[NON_LABELS],
    dummy_labels,
    0.2,
    min_count=3,
    seed=43
)

In [4]:
model = load("model-1.1.0.joblib")
predictions = predict(model, X_test)
# Format correctly in new DataFrame: prediction_df
prediction_df = pd.DataFrame(
    columns=pd.get_dummies(df[LABELS]).columns,
    index=X_test.index,
    data=predictions
)

In [6]:
top_n_pred = top_n_predictions_ids(predictions)
#predicted_sorted = predictions[np.arange(top_n_pred.shape[0]),top_n_pred.T].T
formatted_predictions = format_predictions(X_test, top_n_pred, predictions, prediction_df.columns, model.classes_)
print(formatted_predictions)

[OrderedDict([('label', 'Position_Type_Substitute'), ('probability', '100%')]), OrderedDict([('label', 'Operating_Status_PreK-12 Operating'), ('probability', '100%')]), OrderedDict([('label', 'Object_Type_Benefits'), ('probability', '100%')]), OrderedDict([('label', 'Position_Type_Substitute'), ('probability', '100%')]), OrderedDict([('label', 'Operating_Status_PreK-12 Operating'), ('probability', '100%')]), OrderedDict([('label', 'Pre_K_NO_LABEL'), ('probability', '100%')]), OrderedDict([('label', 'Object_Type_Benefits'), ('probability', '99%')]), OrderedDict([('label', 'Operating_Status_PreK-12 Operating'), ('probability', '99%')]), OrderedDict([('label', 'Pre_K_NO_LABEL'), ('probability', '100%')]), OrderedDict([('label', 'Reporting_Non-School'), ('probability', '100%')]), OrderedDict([('label', 'Object_Type_Other Compensation/Stipend'), ('probability', '100%')]), OrderedDict([('label', 'Sharing_Shared Services'), ('probability', '100%')]), OrderedDict([('label', 'Reporting_Non-Scho